<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-pytorch-step-by-step/Part-III-NLP/06_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Tokenization

We’ll start our NLP journey by following the steps of Alice and Dorothy, from
[Alice’s Adventures in Wonderland](https://ota.bodleian.ox.ac.uk/repository/xmlui/handle/20.500.12024/1476) by Lewis Carroll and [The Wonderful Wizard of Oz](https://ota.bodleian.ox.ac.uk/repository/xmlui/handle/20.500.12024/1740) by L. Frank Baum.

![](https://github.com/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-pytorch-step-by-step/Part-III-NLP/images/alice_dorothy.png?raw=1)

*Left: "Alice and the Baby Pig" illustration by John Tenniel's, from "Alice's Adventure's in Wonderland" (1865).*

*Right: "Dorothy meets the Cowardly Lion" illustration by W.W. Denslow, from "The Wonderful Wizard of Oz" (1900)*

##Setup

In [1]:
%%capture

# # UPDATED
# ###########################################################
!pip install gensim==4.3.1
# # The library has been archived and won't be used anymore
# # # !pip install allennlp==0.9.0
#!pip install flair==0.12.2
#!pip install torchvision==0.15.1
# # # HuggingFace
!pip install transformers==4.32.0
!pip install datasets==2.14.4
# ###########################################################
!pip -q install spacy
!python -m spacy download en_core_web_sm

In [2]:
try:
    import google.colab
    import requests
    url = 'https://raw.githubusercontent.com/dvgodoy/PyTorchStepByStep/master/config.py'
    r = requests.get(url, allow_redirects=True)
    open('config.py', 'wb').write(r.content)
except ModuleNotFoundError:
    pass

from config import *
config_chapter11()
# This is needed to render the plots in this chapter
from plots.chapter11 import *

Finished!


In [3]:
import os
import json
import errno
import requests
import numpy as np
from copy import deepcopy
from operator import itemgetter

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset

from data_generation.nlp import ALICE_URL, WIZARD_URL, download_text
from stepbystep.v4 import StepByStep
# These are the classes we built in Chapter 10
from seq2seq import *

import spacy
import nltk
from nltk.tokenize import sent_tokenize

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import gensim
from gensim import corpora, downloader
from gensim.parsing.preprocessing import *
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [6]:
from datasets import load_dataset, Split
from transformers import (
    DataCollatorForLanguageModeling,
    BertModel, BertTokenizer, BertForSequenceClassification,
    DistilBertModel, DistilBertTokenizer,
    DistilBertForSequenceClassification,
    AutoModelForSequenceClassification,
    AutoModel, AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments, pipeline, TextClassificationPipeline
)
from transformers.pipelines import SUPPORTED_TASKS

##Downloading Books

In [7]:
!rm -rf data

In [8]:
# let's download data
HOME_DIR = "data"
download_text(ALICE_URL, HOME_DIR)
download_text(WIZARD_URL, HOME_DIR)

In [9]:
# let's see the downloaded data
#!cat data/alice28-1476.txt

In [10]:
#!cat data/wizoz10-1740.txt

We need to remove these additions to the original texts:

In [11]:
alice_file = os.path.join(HOME_DIR, "alice28-1476.txt")
with open(alice_file, "r") as f:
  # The actual texts of the books are contained between lines 105 and 3703
  alice_text = "".join(f.readlines()[104:3704])

wizard_file = os.path.join(HOME_DIR, "wizoz10-1740.txt")
with open(wizard_file, "r") as f:
  # The actual texts of the books are contained between lines 309 and 5099
  wizard_text = "".join(f.readlines()[310:5100])

In [12]:
print(alice_text[:500])
print("\n", "#"*70, "\n")
print(wizard_text[:500])

                ALICE'S ADVENTURES IN WONDERLAND

                          Lewis Carroll

               THE MILLENNIUM FULCRUM EDITION 2.8




                            CHAPTER I

                      Down the Rabbit-Hole


  Alice was beginning to get very tired of sitting by her sister
on the bank, and of having nothing to do:  once or twice she had
peeped into the book her sister was reading, but it had no
pictures or conversations in it, `and what is the use of a book,'
thought Alice `w

 ###################################################################### 

                    THE WONDERFUL WIZARD OF OZ


                          1.  The Cyclone


    Dorothy lived in the midst of the great Kansas prairies, with
Uncle Henry, who was a farmer, and Aunt Em, who was the farmer's
wife.  Their house was small, for the lumber to build it had to be
carried by wagon many miles.  There were four walls, a floor and a
roof, which made one room; and this room contained a rusty looking

We can partially automate the removal of the extra lines by setting the real start and end lines of each text in a configuration file.

In [13]:
text_cfg = """fname,start,end
alice28-1476.txt,104,3704
wizoz10-1740.txt,310,5100"""
bytes_written = open(os.path.join(HOME_DIR, 'lines.cfg'), 'w').write(text_cfg)

##Sentence Tokenization

A token is a piece of a text, and to tokenize a text means to split
it into pieces; that is, into a list of tokens.

The most common kind of piece is a word.

So, tokenizing a text usually means to
split it into words using the white space as a separator.

In [14]:
sentence = "I'm following the white rabbit"
tokens = sentence.split(" ")
tokens

["I'm", 'following', 'the', 'white', 'rabbit']

Let's do sentence tokenization, which means to split a text into its sentences.

In [15]:
corpus_alice = sent_tokenize(alice_text)
corpus_wizard = sent_tokenize(wizard_text)

len(corpus_alice), (len(corpus_wizard))

(1612, 2240)

Let’s check one sentence from the first corpus of text.

In [16]:
corpus_alice[2]

'There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!'

Let’s check one sentence from the second corpus of text.

In [17]:
corpus_wizard[30]

'"There\'s a cyclone coming, Em," he called to his wife.'

Our dataset is going to be a collection of CSV files, one file for each book, with each
CSV file containing one sentence per line.

Therefore, we need to:

* clean the line breaks to make sure each sentence is on one line only;
* define an appropriate quote char to "wrap" the sentence such that the original commas and semicolons in the original text do not get misinterpreted as separation chars of the CSV file; and
* add a second column to the CSV file to
identify the original source of the sentence since we’ll be concatenating, and
shuffling the sentences before training a model on our corpora.

The sentence above should end up looking like this:
```log
\"There's a cyclone coming, Em," he called to his wife.\,wizoz10-1740.txt
```

The function below does the grunt work of cleaning, splitting, and saving the
sentences to a CSV file for us:

In [18]:
def sentence_tokenize(source, quote_char="\\", sep_char=",", include_header=True, include_source=True, extensions=("txt"), **kwargs):
  # If source is a folder, goes through all files inside it that match the desired extensions ('txt' by default)
  if os.path.isdir(source):
    filenames = [f for f in os.listdir(source) if os.path.isfile(os.path.join(source, f)) and os.path.splitext(f)[1][1:] in extensions]
  elif isinstance(source, str):
    filenames = [source]

  # If there is a configuration file, builds a dictionary with the corresponding start and end lines of each text file
  config_file = os.path.join(source, "lines.cfg")
  config = {}
  if os.path.exists(config_file):
    with open(config_file, "r") as f:
      rows = f.readlines()
    for r in rows[1:]:
      fname, start, end = r.strip().split(",")
      config.update({fname: (int(start), int(end))})

  new_fnames = []
  # For each file of text
  for fname in filenames:
    # If there's a start and end line for that file, use it
    try:
        start, end = config[fname]
    except KeyError:
        start = None
        end = None

    # Opens the file, slices the configures lines (if any)
    # cleans line breaks and uses the sentence tokenizer
    with open(os.path.join(source, fname), 'r') as f:
        contents = (''.join(f.readlines()[slice(start, end, None)]).replace('\n', ' ').replace('\r', ''))
    corpus = sent_tokenize(contents, **kwargs)

    # Builds a CSV file containing tokenized sentences
    base = os.path.splitext(fname)[0]
    new_fname = f'{base}.sent.csv'
    new_fname = os.path.join(source, new_fname)
    with open(new_fname, 'w') as f:
        # Header of the file
        if include_header:
            if include_source:
                f.write('sentence,source\n')
            else:
                f.write('sentence\n')
        # Writes one line for each sentence
        for sentence in corpus:
            if include_source:
                f.write(f'{quote_char}{sentence}{quote_char}{sep_char}{fname}\n')
            else:
                f.write(f'{quote_char}{sentence}{quote_char}\n')
    new_fnames.append(new_fname)

  # Returns list of the newly generated CSV files
  return sorted(new_fnames)

In [19]:
new_fnames = sentence_tokenize(HOME_DIR)
new_fnames

['data/alice28-1476.sent.csv', 'data/wizoz10-1740.sent.csv']

##Spacy sentence tokenization

In [20]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

sentences = []
for doc in nlp.pipe(corpus_alice):
  sentences.extend(sent.text for sent in doc.sents)

len(sentences), sentences[2]

(1615,
 'There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!')

##HuggingFace’s Dataset

In [21]:
# let's load from local files using HuggingFace
dataset = load_dataset(path="csv", data_files=new_fnames, quotechar="\\", split=Split.TRAIN)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
# let's see attributes, like features, num_columns, and shape
dataset.features, dataset.num_columns, dataset.shape

({'sentence': Value(dtype='string', id=None),
  'source': Value(dtype='string', id=None)},
 2,
 (3852, 2))

In [23]:
dataset[2]

{'sentence': 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!',
 'source': 'alice28-1476.txt'}

In [24]:
dataset["sentence"][:3]

["                ALICE'S ADVENTURES IN WONDERLAND                            Lewis Carroll                 THE MILLENNIUM FULCRUM EDITION 2.8                                 CHAPTER I                        Down the Rabbit-Hole     Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do:  once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'",
 'So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.',
 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!']

In [25]:
dataset["source"][:3]

['alice28-1476.txt', 'alice28-1476.txt', 'alice28-1476.txt']

In [26]:
# check the unique sources
dataset.unique("source")

['alice28-1476.txt', 'wizoz10-1740.txt']

In [27]:
# let's create new columns
def is_alice_label(row):
  is_alice = int(row["source"] == "alice28-1476.txt")
  return {"label": is_alice}

In [28]:
dataset = dataset.map(is_alice_label)

Map:   0%|          | 0/3852 [00:00<?, ? examples/s]

In [29]:
dataset[2]

{'sentence': 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!',
 'source': 'alice28-1476.txt',
 'label': 1}

In [30]:
# Now, we can finally shuffle the dataset and split it into training and test sets
shuffled_dataset = dataset.shuffle(seed=42)
split_dataset = shuffled_dataset.train_test_split(test_size=0.2)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'source', 'label'],
        num_rows: 3081
    })
    test: Dataset({
        features: ['sentence', 'source', 'label'],
        num_rows: 771
    })
})

In [31]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [32]:
train_dataset[0]

{'sentence': "Why, there's hardly enough of me left to make ONE respectable person!'",
 'source': 'alice28-1476.txt',
 'label': 1}

##Word Tokenization

In [33]:
# naïve word tokenization
sentence = "I'm following the white rabbit"
tokens = sentence.split(" ")
tokens

["I'm", 'following', 'the', 'white', 'rabbit']

In [34]:
# Let’s try using Gensim
preprocess_string(sentence)

['follow', 'white', 'rabbit']

In [35]:
# We won’t be removing stop words or performing stemming here
filters = [lambda x: x.lower(),
           strip_tags,
           strip_punctuation,
           strip_multiple_whitespaces,
           strip_numeric]

preprocess_string(sentence, filters=filters)

['i', 'm', 'following', 'the', 'white', 'rabbit']

In [36]:
# Another option
tokens = simple_preprocess(sentence)
tokens

['following', 'the', 'white', 'rabbit']

##Vocabulary

In [37]:
sentences = train_dataset["sentence"]
tokens = [simple_preprocess(sent) for sent in sentences]
tokens[0]

['why',
 'there',
 'hardly',
 'enough',
 'of',
 'me',
 'left',
 'to',
 'make',
 'one',
 'respectable',
 'person']

In [38]:
# build a vocabulary
dictionary = corpora.Dictionary(tokens)
print(dictionary)

Dictionary<3684 unique tokens: ['enough', 'hardly', 'left', 'make', 'me']...>


In [39]:
dictionary.num_docs

3081

In [40]:
dictionary.num_pos

50331

In [41]:
dictionary.token2id

{'enough': 0,
 'hardly': 1,
 'left': 2,
 'make': 3,
 'me': 4,
 'of': 5,
 'one': 6,
 'person': 7,
 'respectable': 8,
 'there': 9,
 'to': 10,
 'why': 11,
 'hatter': 12,
 'muttered': 13,
 'the': 14,
 'and': 15,
 'at': 16,
 'beyond': 17,
 'boy': 18,
 'found': 19,
 'handsome': 20,
 'his': 21,
 'however': 22,
 'last': 23,
 'manly': 24,
 'she': 25,
 'was': 26,
 'who': 27,
 'wise': 28,
 'years': 29,
 'course': 30,
 'it': 31,
 'mock': 32,
 'said': 33,
 'turtle': 34,
 'anything': 35,
 'bottle': 36,
 'does': 37,
 'drink': 38,
 'eat': 39,
 'happen': 40,
 'herself': 41,
 'interesting': 42,
 'is': 43,
 'just': 44,
 'know': 45,
 'll': 46,
 'or': 47,
 'see': 48,
 'so': 49,
 'something': 50,
 'sure': 51,
 'this': 52,
 'what': 53,
 'whenever': 54,
 'alternately': 55,
 'as': 56,
 'baby': 57,
 'duchess': 58,
 'even': 59,
 'for': 60,
 'howling': 61,
 'moment': 62,
 'occasionally': 63,
 'pause': 64,
 'sneezed': 65,
 'sneezing': 66,
 'without': 67,
 'but': 68,
 'clouds': 69,
 'come': 70,
 'from': 71,
 'great

In [42]:
vocab = list(dictionary.token2id.keys())
vocab[:5]

['enough', 'hardly', 'left', 'make', 'me']

In [43]:
# collection frequencies
dictionary.cfs

{11: 67,
 9: 180,
 1: 12,
 0: 24,
 5: 1078,
 4: 210,
 2: 38,
 10: 1470,
 3: 56,
 6: 176,
 8: 1,
 7: 7,
 13: 1,
 14: 3669,
 12: 45,
 16: 375,
 23: 59,
 22: 28,
 25: 718,
 19: 59,
 18: 5,
 27: 138,
 26: 688,
 20: 4,
 15: 1974,
 24: 2,
 28: 7,
 17: 4,
 21: 245,
 29: 10,
 30: 35,
 31: 811,
 33: 645,
 32: 44,
 34: 46,
 45: 118,
 50: 25,
 42: 4,
 43: 293,
 51: 37,
 40: 9,
 41: 70,
 54: 7,
 39: 21,
 47: 94,
 38: 6,
 35: 28,
 49: 365,
 46: 61,
 44: 75,
 48: 129,
 53: 190,
 52: 227,
 36: 7,
 37: 12,
 59: 35,
 58: 30,
 65: 1,
 63: 1,
 56: 458,
 60: 376,
 57: 12,
 66: 5,
 61: 2,
 55: 1,
 67: 28,
 62: 36,
 64: 2,
 68: 379,
 75: 15,
 73: 640,
 74: 4,
 78: 16,
 76: 59,
 77: 7,
 70: 89,
 71: 121,
 69: 4,
 79: 100,
 72: 138,
 80: 24,
 84: 96,
 82: 17,
 92: 13,
 96: 732,
 94: 176,
 87: 70,
 89: 18,
 83: 78,
 86: 37,
 90: 8,
 85: 121,
 81: 32,
 95: 243,
 88: 181,
 91: 18,
 93: 4,
 109: 456,
 106: 39,
 103: 14,
 107: 15,
 101: 6,
 111: 57,
 112: 28,
 100: 5,
 104: 2,
 108: 19,
 105: 58,
 99: 98,
 110: 76

In [44]:
# document frequencies
dictionary.dfs

{11: 67,
 9: 176,
 1: 12,
 0: 24,
 5: 816,
 4: 185,
 2: 38,
 10: 1078,
 3: 54,
 6: 162,
 8: 1,
 7: 7,
 13: 1,
 14: 1896,
 12: 45,
 16: 336,
 23: 59,
 22: 28,
 25: 481,
 19: 56,
 18: 5,
 27: 133,
 26: 547,
 20: 4,
 15: 1272,
 24: 2,
 28: 7,
 17: 4,
 21: 200,
 29: 9,
 30: 35,
 31: 617,
 33: 636,
 32: 44,
 34: 45,
 45: 113,
 50: 25,
 42: 4,
 43: 263,
 51: 37,
 40: 9,
 41: 65,
 54: 7,
 39: 20,
 47: 85,
 38: 6,
 35: 28,
 49: 348,
 46: 56,
 44: 75,
 48: 123,
 53: 183,
 52: 223,
 36: 7,
 37: 12,
 59: 35,
 58: 29,
 65: 1,
 63: 1,
 56: 327,
 60: 346,
 57: 12,
 66: 5,
 61: 2,
 55: 1,
 67: 27,
 62: 34,
 64: 2,
 68: 368,
 75: 15,
 73: 554,
 74: 4,
 78: 16,
 76: 59,
 77: 7,
 70: 86,
 71: 119,
 69: 4,
 79: 100,
 72: 135,
 80: 24,
 84: 89,
 82: 17,
 92: 12,
 96: 554,
 94: 166,
 87: 70,
 89: 18,
 83: 78,
 86: 36,
 90: 8,
 85: 113,
 81: 32,
 95: 210,
 88: 176,
 91: 18,
 93: 3,
 109: 346,
 106: 39,
 103: 14,
 107: 15,
 101: 6,
 111: 57,
 112: 28,
 100: 5,
 104: 2,
 108: 19,
 105: 58,
 99: 98,
 110: 76,


In [45]:
# doc to id
sentence = "follow the white rabbit"
new_tokens = simple_preprocess(sentence)
ids = dictionary.doc2idx(new_tokens)
print(new_tokens)
print(ids)

['follow', 'the', 'white', 'rabbit']
[761, 14, 896, 475]


In [46]:
# add special token to unknown word
special_tokens = {
    "[PAD]": 0,
    "[UNK]": 1
}

dictionary.patch_with_special_tokens(special_tokens)

In [47]:
def get_rare_ids(dictionary, min_freq):
  rare_ids = [t[0] for t in dictionary.cfs.items() if t[1] < min_freq]
  return rare_ids

In [48]:
def make_vocab(sentences, folder=None, special_tokens=None, vocab_size=None, min_freq=None):

  if folder is not None:
    if not os.path.exists(folder):
      os.mkdir(folder)

  # tokenizes the sentences and create a Dictionary
  tokens = [simple_preprocess(sent) for sent in sentences]
  dictionary = corpora.Dictionary(tokens)

  # keeps only the most frequent words (vocab size)
  if vocab_size is not None:
    dictionary.filter_extremes(keep_n=vocab_size)

  # removes rare words (in case the vocab size still includes words with low frequency)
  if min_freq is not None:
    rare_tokens = get_rare_ids(dictionary, min_freq)
    dictionary.filter_tokens(bad_ids=rare_tokens)

  # gets the whole list of tokens and frequencies
  items = dictionary.cfs.items()

  # sorts the tokens in descending order
  words = [dictionary[t[0]] for t in sorted(dictionary.cfs.items(), key=lambda t: -t[1])]

  # prepends special tokens, if any
  if special_tokens is not None:
    to_add = []
    for special_token in special_tokens:
      if special_token not in words:
        to_add.append(special_token)
    words = to_add + words

  with open(os.path.join(folder, "vocab.txt"), "w") as f:
    for word in words:
      f.write(f"{word}\n")

In [49]:
make_vocab(
    train_dataset["sentence"],
    "our_vocab/",
    special_tokens=['[PAD]', '[UNK]', '[SEP]', '[CLS]', '[MASK]'],
    min_freq=2
)

## HugginFace's Tokenizer

So, before loading a pre-trained tokenizer, let’s create our own tokenizer using our
own vocabulary.

In [50]:
tokenizer = BertTokenizer("our_vocab/vocab.txt")

In [51]:
# Then, let’s tokenize a new sentence
new_sentence = "follow the white rabbit neo"
new_tokens = tokenizer.tokenize(new_sentence)
new_tokens

['follow', 'the', 'white', 'rabbit', '[UNK]']

In [52]:
# get the indices
new_ids = tokenizer.convert_tokens_to_ids(new_tokens)
new_ids

[1172, 5, 186, 228, 1]

In [53]:
# perform two steps at once
new_ids = tokenizer.encode(new_sentence)
new_ids

[3, 1172, 5, 186, 228, 1, 2]

In [54]:
# check the extra tokens
tokenizer.convert_ids_to_tokens(new_ids)

['[CLS]', 'follow', 'the', 'white', 'rabbit', '[UNK]', '[SEP]']

In [55]:
# get rid of the special tokens
tokenizer.encode(new_sentence, add_special_tokens=False)

[1172, 5, 186, 228, 1]

In [56]:
# we can simply call the tokenizer itself
tokenizer(new_sentence, add_special_tokens=False, return_tensors="pt")

{'input_ids': tensor([[1172,    5,  186,  228,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [57]:
# token_type_ids only makes sense if the input has more than one sentence
sentence1 = "follow the white rabbit neo"
sentence2 = "no one can be told what the matrix is"
joined_sentences = tokenizer(sentence1, sentence2)
joined_sentences

{'input_ids': [3, 1172, 5, 186, 228, 1, 2, 52, 48, 77, 33, 309, 40, 5, 1, 29, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [58]:
# Let’s convert the IDs back to tokens and inspect the result
print(tokenizer.convert_ids_to_tokens(joined_sentences["input_ids"]))

['[CLS]', 'follow', 'the', 'white', 'rabbit', '[UNK]', '[SEP]', 'no', 'one', 'can', 'be', 'told', 'what', 'the', '[UNK]', 'is', '[SEP]']


In [59]:
# we may pad the sequences to get them all with the same length
separate_sentences = tokenizer([sentence1, sentence2], padding=True)
separate_sentences

{'input_ids': [[3, 1172, 5, 186, 228, 1, 2, 0, 0, 0, 0], [3, 52, 48, 77, 33, 309, 40, 5, 1, 29, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [60]:
# Let’s convert the IDs back to tokens again
print(tokenizer.convert_ids_to_tokens(separate_sentences["input_ids"][0]))
print(separate_sentences["attention_mask"][0])

['[CLS]', 'follow', 'the', 'white', 'rabbit', '[UNK]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]


In [61]:
# let's create two batches
first_sentences = [sentence1, "another first sentence"]
second_sentences = [sentence2, "a second sentence here"]

batch_of_pairs = tokenizer(first_sentences, second_sentences)
first_input = tokenizer.convert_ids_to_tokens(batch_of_pairs["input_ids"][0])
second_input = tokenizer.convert_ids_to_tokens(batch_of_pairs["input_ids"][1])

print(first_input)
print(second_input)

['[CLS]', 'follow', 'the', 'white', 'rabbit', '[UNK]', '[SEP]', 'no', 'one', 'can', 'be', 'told', 'what', 'the', '[UNK]', 'is', '[SEP]']
['[CLS]', 'another', 'first', 'sentence', '[SEP]', '[UNK]', 'second', 'sentence', 'here', '[SEP]']


Finally, let’s apply our tokenizer to our dataset of sentences, padding them and
returning PyTorch tensors:

In [62]:
tokenized_dataset = tokenizer(dataset["sentence"],
                              padding=True,
                              return_tensors="pt",
                              max_length=50,
                              truncation=True)
tokenized_dataset["input_ids"]

tensor([[   3,   27,    1,  ...,   92,  794,    2],
        [   3,   23,   11,  ..., 1355,    5,    2],
        [   3,   46,   12,  ...,    0,    0,    0],
        ...,
        [   3,    1,    6,  ...,    0,    0,    0],
        [   3,    6,  125,  ...,    0,    0,    0],
        [   3,    1,    1,  ...,    0,    0,    0]])

Can we feed the input_ids to BERT and watch the magic happen?

BERT is actually using vectors to represent the words. The
token IDs we’ll be sending it are simply the indices of an enormous lookup table.

That lookup table has a very nice name: **Word Embeddings**.

Each row of the lookup table corresponds to a different token, and each row is
represented by a vector. The size of the vectors is the dimensionality of the
embedding.